### Importing Libraries

In [1]:
import osgeo
from osgeo import gdal
import rasterio
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
for gpu in gpus:
    print(gpu)
import numpy as np
import matplotlib 
from matplotlib import pyplot as plt
import re
    
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU,  Dense, Flatten, Reshape, Dropout, UpSampling2D, Lambda
from tensorflow.keras.preprocessing import image
from PIL import Image
import os

In [2]:
file_path = "images_downscaled\\April_7_2022.tif"
dataset = rasterio.open(file_path)

# Print general information
print("Raster Information:")
print(f"Number of Bands: {dataset.count}")
print(f"Width: {dataset.width}")
print(f"Height: {dataset.height}")
print(f"Coordinate Reference System (CRS): {dataset.crs}")
print(f"Transform (Affine): {dataset.transform}")

Raster Information:
Number of Bands: 4
Width: 256
Height: 256
Coordinate Reference System (CRS): None
Transform (Affine): | 1.00, 0.00, 0.00|
| 0.00, 1.00, 0.00|
| 0.00, 0.00, 1.00|


c:\Users\remne\AppData\Local\Programs\Python\Python311\Lib\site-packages\rasterio\__init__.py:319: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Downscaling the Images to a (256, 256) Dimension.

In [3]:
input_folder = "images"
output_folder = "images_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

Downscaling the Masks to a (256, 256) Dimension.

In [4]:
input_folder = "masks"
output_folder = "masks_downscaled"
target_resolution = (256, 256)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate over the files in the input folder
for file_name in os.listdir(input_folder):
    # Check if the file is an image (you can modify the condition based on your image file extensions)
    if file_name.endswith(".tif"):
        # Construct the full paths to the input and output images
        input_image_path = os.path.join(input_folder, file_name)
        output_image_path = os.path.join(output_folder, file_name)

        # Open and resize the input image
        input_image = Image.open(input_image_path)
        resized_image = input_image.resize(target_resolution)

        # Save the resized image to the output folder
        resized_image.save(output_image_path)

In [28]:
# def build_generator():

#     model = Sequential()

#     model.add(Dense(256*256, input_dim = 100))
#     model.add(LeakyReLU(0.2))
#     model.add(Reshape((256, 256, 1))) # 1 is the number of channels

#     # Upsampling Block 1
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Upsampling Block 2
#     model.add(UpSampling2D())
#     model.add(Conv2D(64, 3, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional Block 1
#     model.add(Conv2D(64, 2, padding='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutuonal Block 2
#     model.add(Conv2D(64, 2, padding ='same'))
#     model.add(LeakyReLU(0.2))

#     # Convolutional layer to get to one channel
#     model.add(Conv2D(1, 1, padding='same', activation='sigmoid'))


#     return model

def build_generator():
    model = Sequential()

    model.add(Dense(64 * 64 * 1, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((64, 64, 1)))

    # Upsampling Block 1
    model.add(UpSampling2D())
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # Upsampling Block 2
    model.add(UpSampling2D())
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional Block 1
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # Convolutional Block 2
    model.add(Conv2D(64, 3, padding='same'))
    model.add(LeakyReLU(0.2))

    # # Convolutional layer to get to one channel
    # model.add(Conv2D(1, 3, padding='same', activation='sigmoid'))

    # Convolutional layer to get to four channels
    model.add(Conv2D(4, 3, padding='same', activation='sigmoid'))


    return model



generator = build_generator()

generator.summary()



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 4096)              413696    
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 4096)              0         
                                                                 
 reshape_3 (Reshape)         (None, 64, 64, 1)         0         
                                                                 
 up_sampling2d_6 (UpSampling  (None, 128, 128, 1)      0         
 2D)                                                             
                                                                 
 conv2d_27 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 128, 128, 64)      0         
                                                      

In [36]:
# def build_discriminator():

#     model = Sequential()

#     # Convolutional Block 1
#     model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(256, 256, 1)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 2
#     model.add(Conv2D(128, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 3
#     model.add(Conv2D(256, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Convolutional Block 4
#     model.add(Conv2D(512, (3, 3)))
#     model.add(LeakyReLU(0.2))
#     model.add(Dropout(0.25))

#     # Flatten and then Condense into 1
#     model.add(Flatten())
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))

#     return model


def build_discriminator():
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(64, (3, 3), strides=(2, 2), input_shape=(256, 256, 1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 2
    model.add(Conv2D(128, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 3
    model.add(Conv2D(256, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Convolutional Block 4
    model.add(Conv2D(512, (3, 3)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.25))

    # Flatten and then Condense into 1
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [37]:
discriminator = build_discriminator()

In [38]:
discriminator.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 127, 127, 64)      640       
                                                                 
 leaky_re_lu_36 (LeakyReLU)  (None, 127, 127, 64)      0         
                                                                 
 dropout_20 (Dropout)        (None, 127, 127, 64)      0         
                                                                 
 conv2d_37 (Conv2D)          (None, 125, 125, 128)     73856     
                                                                 
 leaky_re_lu_37 (LeakyReLU)  (None, 125, 125, 128)     0         
                                                                 
 dropout_21 (Dropout)        (None, 125, 125, 128)     0         
                                                                 
 conv2d_38 (Conv2D)          (None, 123, 123, 256)    

In [39]:
# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

In [40]:
# Get the list of image and mask files
image_folder = "images_downscaled"
mask_folder = "masks_downscaled"
image_files = os.listdir(image_folder)
mask_files = os.listdir(mask_folder)

# Set the number of training iterations
epochs = 100
batch_size = 8

In [41]:
def save_generated_images(epoch):
    # Generate fake images
    noise = np.random.normal(0, 1, (10, 100))
    fake_images = generator.predict(noise)

    # Create a directory to save the generated images
    save_dir = "generated_images"
    os.makedirs(save_dir, exist_ok=True)

    # Save the generated images
    for i, image in enumerate(fake_images):
        filename = f"generated_image_{epoch}_{i}.png"
        filepath = os.path.join(save_dir, filename)
        image = (image * 255).astype(np.uint8)
        Image.fromarray(image).save(filepath)

In [42]:
# Training loop
for epoch in range(epochs):
    # ---------------------
    # Train the discriminator
    # ---------------------
    # Select a random batch of images and masks
    idx = np.random.randint(0, len(image_files), batch_size)
    selected_image_files = [image_files[i] for i in idx]
    selected_mask_files = [mask_files[i] for i in idx]

    # Load and preprocess the real images and masks
    real_images = []
    real_masks = []
    for image_file, mask_file in zip(selected_image_files, selected_mask_files):
        image_path = os.path.join(image_folder, image_file)
        mask_path = os.path.join(mask_folder, mask_file)
        image = np.array(Image.open(image_path))
        mask = np.array(Image.open(mask_path))
        real_images.append(image)
        real_masks.append(mask)
    real_images = np.array(real_images)
    real_masks = np.array(real_masks)

    # Generate fake images
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)

    # Create a combined batch of real and fake images
    x = np.concatenate((real_images, fake_images))
    # Create labels for the discriminator (1 for real images, 0 for fake images)
    y = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))))

    # Train the discriminator
    discriminator_loss = discriminator.train_on_batch(x, y)

    # ---------------------
    # Train the generator
    # ---------------------
    # Generate a new batch of noise
    noise = np.random.normal(0, 1, (batch_size, 100))
    # Create labels for the generator (1 for all generated images, as we want the discriminator to classify them as real)
    y = np.ones((batch_size, 1))

    # Train the generator
    generator_loss = combined_model.train_on_batch(noise, y)

    # Print the losses
    print(f"Epoch {epoch+1}/{epochs} - Discriminator Loss: {discriminator_loss} - Generator Loss: {generator_loss}")

    # Save generated images at specified intervals
    if (epoch+1) % 10 == 0:
        save_generated_images(epoch+1)

1/1 [==============================] - 0s 182ms/step


ValueError: in user code:

    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\remne\AppData\Roaming\Python\Python311\site-packages\keras\